Load Dataset

In [ ]:
# load benign image data

import pickle
with open('Cleaned_Breast_Types/benign_list.pkl', 'rb') as file:
    loaded_img_list = pickle.load(file)
    
normal_list=[]
x = 0
while True:
  try:
    array_a_loaded = loaded_img_list[x]
    normal_list.append(array_a_loaded)
    x+=1
  except:
    print('end!')
    break
print(x)

In [ ]:
# load cancer image data

import pickle
with open('Cleaned_Breast_Types/cancer_list.pkl', 'rb') as file:
    loaded_img_list = pickle.load(file)
    
cancer_list=[]
x = 0
while True:
  try:
    array_a_loaded = loaded_img_list[x]
    cancer_list.append(array_a_loaded)
    x+=1
  except:
    print('end!')
    break
print(x)

In [ ]:
# load normal image data

import pickle
with open('Cleaned_Breast_Types/normal_list.pkl', 'rb') as file:
    loaded_img_list = pickle.load(file)
    
cancer_list=[]
x = 0
while True:
  try:
    array_a_loaded = loaded_img_list[x]
    cancer_list.append(array_a_loaded)
    x+=1
  except:
    print('end!')
    break
print(x)

Split Dataset (Normal and Cancer only)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Combine the lists and create labels (normal and cancer)
X = np.array(normal_list + cancer_list)
y = np.array([0] * len(normal_list) + [1] * len(cancer_list))

# train 0.8, test 0.1, val 0.1
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=42,stratify=y)
X_test,X_valid,y_test,y_valid = train_test_split(X_test,y_test,test_size=0.5,shuffle=True,random_state=42)

In [ ]:
X_train.shape

CNN - LSTM (1st Architecture)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils import shuffle

# Adjust the input shape to include the number of channels (e.g., grayscale images have 1 channel)
input_shape = (256, 256, 3)

def build_cnn_lstm_model(input_shape):
    model = Sequential()
 
    model.add(Conv2D(32, (3, 3), activation='relu', 
                     input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
 
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))
 
    model.add(Dense(1, activation='sigmoid'))
    return model
 
model1 = build_cnn_lstm_model(input_shape)
model1.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

# Convert your data to tf.data.Dataset for better performance
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

# Train the model
history1 = model1.fit(
    train_dataset,
    epochs=20,
    batch_size=32,
    validation_data=valid_dataset
    # callbacks=[tensorboard_callback]
)

# Evaluate the model on the valid set
test_loss, test_acc = model1.evaluate(valid_dataset)
print(f'Test accuracy: {test_acc:.4f}')

Confusion Matrix and Classification Report (1st Architecture)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Predict probabilities for the validation set
predictions = model1.predict(valid_dataset)
predicted_labels = (predictions > 0.5).astype("int32")  # Apply a threshold to get binary labels

# Calculate accuracy
accuracy = accuracy_score(y_valid, predicted_labels)
print(f'Validation accuracy: {accuracy:.4f}')

# Generate a classification report
print(classification_report(y_valid, predicted_labels, digits=5))

# Generate a confusion matrix
cm = confusion_matrix(y_valid, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

Learning Curve (1st Architecture)

In [ ]:
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
 
# summarize history for loss
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

CNN - LSTM (2nd Architecture)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils import shuffle

# Adjust the input shape to include the number of channels (e.g., grayscale images have 1 channel)
input_shape = (256, 256, 3)

def build_cnn_lstm_model(input_shape):
    model = Sequential()
 
    model.add(Conv2D(32, (3, 3), activation='relu', 
                     input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

 
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
 
model2 = build_cnn_lstm_model(input_shape)
model2.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

# Convert your data to tf.data.Dataset for better performance
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

# Train the model
history2 = model2.fit(
    train_dataset,
    epochs=20,
    batch_size=32,
    validation_data=valid_dataset
    # callbacks=[tensorboard_callback]
)

# Evaluate the model on the valid set
test_loss, test_acc = model2.evaluate(valid_dataset)
print(f'Test accuracy: {test_acc:.4f}')

Confusion Matrix and Classification Report (2nd Architecture)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Predict probabilities for the validation set
predictions = model2.predict(valid_dataset)
predicted_labels = (predictions > 0.5).astype("int32")  # Apply a threshold to get binary labels

# Calculate accuracy
accuracy = accuracy_score(y_valid, predicted_labels)
print(f'Validation accuracy: {accuracy:.4f}')

# Generate a classification report
print(classification_report(y_valid, predicted_labels, digits=5))

# Generate a confusion matrix
cm = confusion_matrix(y_valid, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

Learning Curve (2nd Architecture)

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
 
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

CNN - LSTM (3rd Architecture)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils import shuffle

# Adjust the input shape to include the number of channels (e.g., grayscale images have 1 channel)
input_shape = (256, 256, 3)

def build_cnn_lstm_model(input_shape):
    model = Sequential()
 
    model.add(Conv2D(32, (3, 3), activation='relu', 
                     input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
 
model3 = build_cnn_lstm_model(input_shape)
model3.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model3.summary()

# Convert your data to tf.data.Dataset for better performance
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

# Train the model
history3 = model3.fit(
    train_dataset,
    epochs=20,
    batch_size=32,
    validation_data=valid_dataset
    # callbacks=[tensorboard_callback]
)

# Evaluate the model on the valid set
test_loss, test_acc = model3.evaluate(valid_dataset)
print(f'Test accuracy: {test_acc:.4f}')

Confusion Matrix and Classification Report (3rd Architecture)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Predict probabilities for the validation set
predictions = model3.predict(valid_dataset)
predicted_labels = (predictions > 0.5).astype("int32")  # Apply a threshold to get binary labels

# Calculate accuracy
accuracy = accuracy_score(y_valid, predicted_labels)
print(f'Validation accuracy: {accuracy:.4f}')

# Generate a classification report
print(classification_report(y_valid, predicted_labels, digits=5))

# Generate a confusion matrix
cm = confusion_matrix(y_valid, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

Learning Curve (3rd Architecture)

In [ ]:
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
 
# summarize history for loss
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Model Tuning (Random Search)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dropout, Dense
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters
from tensorflow.keras.callbacks import EarlyStopping


def build_cnn_lstm_model(hp):
    model = Sequential()
 
    model.add(Conv2D(filters=hp.Int('filters1', min_value=32, max_value=128, step=32),
                     kernel_size=(3,3),
                     activation='relu', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(filters=hp.Int('filters2', min_value=32, max_value=128, step=32),
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(filters=hp.Int('filters3', min_value=32, max_value=128, step=32),
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(filters=hp.Int('filters4', min_value=32, max_value=128, step=32),
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(units=hp.Int('unit', min_value=32, max_value=128, step=32), return_sequences=False))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))
   
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy', metrics=['accuracy'])
   
    return model

# Define the hyperparameter space for epochs and batch size
hp = HyperParameters()
hp.Int('epochs', min_value=10, max_value=50, step=10)  # Range of epochs
hp.Int('batch_size', min_value=16, max_value=64, step=16)  # Range of batch sizes

# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Create the RandomSearch tuner
tuner = RandomSearch(
    build_cnn_lstm_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='/kaggle/working/',
    project_name='cnnlstmtuning',
    hyperparameters=hp
)

# Perform the search
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid))

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

Print the best hyperparameters

In [ ]:
print("Filter 1: ", best_hps.get('filters1'))
print("Filter 2: ", best_hps.get('filters2'))
print("Filter 3: ", best_hps.get('filters3'))
print("Filter 4: ", best_hps.get('filters4'))
print("Unit: ", best_hps.get('unit'))
print("Dropout Rate: ", best_hps.get('dropout_rate'))
print("LR: ", best_hps.get('learning_rate'))
print("Epochs: ", best_hps.get('epochs'))
print("BS: ", best_hps.get('batch_size'))

Rebuild CNN - LSTM Model (2nd Architecture) using Best Parameters

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dropout, Dense
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters
from tensorflow.keras.callbacks import EarlyStopping
# Adjust the input shape to include the number of channels (e.g., grayscale images have 1 channel)
input_shape = (256, 256, 3)

def build_cnn_lstm_model(input_shape):
    model = Sequential()
 
    model.add(Conv2D(96, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
 
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.3))
 
    model.add(Dense(1, activation='sigmoid'))
    return model
 
model = build_cnn_lstm_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Convert your data to tf.data.Dataset for better performance
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

 
# Train the model
history1 = model.fit(
    train_dataset,
    epochs=40,
    batch_size=32,
    validation_data=valid_dataset,
)

# Evaluate the model on the valid set
test_loss, test_acc = model.evaluate(valid_dataset)
print(f'Test accuracy: {test_acc:.4f}')

Confusion Matrix and Classification Report (Tuned Model)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Predict probabilities for the validation set
predictions = model.predict(valid_dataset)
predicted_labels = (predictions > 0.5).astype("int32")  # Apply a threshold to get binary labels

# Calculate accuracy
accuracy = accuracy_score(y_valid, predicted_labels)
print(f'Validation accuracy: {accuracy:.4f}')

# Generate a classification report
print(classification_report(y_valid, predicted_labels, digits=5))

# Generate a confusion matrix
cm = confusion_matrix(y_valid, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

Learning Curve (Tuned Model)

In [ ]:
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
 
# summarize history for loss
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()